### Extracting Text from Resumes
In this file, i tried to use NER and Regualr Experessions to extract skills. education, experience etc.

In [8]:
import re
import spacy
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [50]:
# Load spaCy model
nlp = spacy.load("en_core_web_lg")

In [51]:
# Load sentence transformer model
sentence_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [62]:
yeshi = """Yeshaswi Prakash
91 90254 16053 
yeshaswiprakash.com
linkedin.com/in/yeshaswi
github.com/yeshaswi

Profile Summary
Software Developer | 1 YOE | 4× Hackathon Winner | 3rd Year CSE Student at SRMIST
Experienced in building AI and ML-powered applications, leading teams, and contributing to impactful projects. Actively involved in tech communities and campus leadership roles.

Experience
HeadStarter AI – Remote
Software Engineering Fellow
Jul 2024 – Sep 2024

Built AI applications using Next.js, OpenAI, Pinecone, and Stripe API.

Led a team of 3 engineering fellows.

Mentored by engineers from Amazon, Bloomberg, and Capital One.

Tiger Analytics – Chennai
Software Engineering Intern
Apr 2024 – Jul 2024

Developed an ML recommendation system for an e-commerce client.

Conducted EDA and created visualizations to uncover key business insights.

Rotork – Chennai
Software Engineering Intern
Mar 2024 – Apr 2024

Developed a Python desktop application with a simple UI for remote actuator software updates.

Enabled non-technical users to perform updates without on-site visits.

Education
SRM Institute of Science and Technology – Chennai
B.Tech in Computer Science and Engineering
Sep 2022 – May 2026

CGPA: 9.02

Leadership Roles:

Secretary, CSI Club

Joint Treasurer, ACE Club

Director of Design, NL Editorial Board

D.A.V. Group of Schools (CBSE) – Chennai
2008 – 2022

Percentage: 82.4%

Projects
CourseGPT (Next.js, TypeScript, Firebase)

Built an AI-powered web app to auto-generate editable course content and video scripts.

Integrated Google Gemini and YouTube APIs.

Features a Firebase backend and retro-inspired UI.

Project Demo / Details on Portfolio

Skills
Languages
Python (4 yrs), SQL (4 yrs), JavaScript (2 yrs), C++ (1 yr), C (1 yr), PHP (1 yr)

Tools & Technologies
React, TypeScript, Jupyter Notebook, Bootstrap, Git, Firebase, Figma, Adobe Photoshop, Spline

Creative & Soft Skills
Graphic Design, UI/UX Design, Event Coordination, Leadership, Teamwork

Awards
1st Place – Hack the Horizon (University Feature)
1st Place – Hackonomics 2025 (University Feature)
2nd Place – EDHacks 2024 (University Feature)
3rd Place – Ideathon 4.0 2024 (University Feature)

Volunteering
Kalaiselvi Karunalaya Welfare Society – Chennai
Summer 2024

Volunteered at a combined orphanage and old age home."""

In [68]:
#preprocess text
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'\S+@\S+', '', text)
    text = re.sub(r'\b\d{3}[-.]?\d{3}[-.]?\d{4}\b', '', text)
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

In [64]:
print(preprocess_text(yeshi))
ntext = preprocess_text(yeshi)

yeshaswi prakash 91 90254 16053 yeshaswiprakash com linkedin com in yeshaswi github com yeshaswi profile summary software developer 1 yoe 4 hackathon winner 3rd year cse student at srmist experienced in building ai and ml powered applications leading teams and contributing to impactful projects actively involved in tech communities and campus leadership roles experience headstarter ai remote software engineering fellow jul 2024 sep 2024 built ai applications using next js openai pinecone and stripe api led a team of 3 engineering fellows mentored by engineers from amazon bloomberg and capital one tiger analytics chennai software engineering intern apr 2024 jul 2024 developed an ml recommendation system for an e commerce client conducted eda and created visualizations to uncover key business insights rotork chennai software engineering intern mar 2024 apr 2024 developed a python desktop application with a simple ui for remote actuator software updates enabled non technical users to perf

### SKILLLSSS

In [69]:
def extract_skills(text, skill_patterns=None):
    
    common_skills = [
        "python", "java", "javascript", "c++", "c#", "ruby", "php", "swift", "kotlin",
        "react", "angular", "vue", "django", "flask", "spring", "node.js", "express",
        "tensorflow", "pytorch", "scikit-learn", "pandas", "numpy", "matplotlib",
        "sql", "mysql", "postgresql", "mongodb", "oracle", "firebase", "aws", "azure",
        "gcp", "docker", "kubernetes", "jenkins", "git", "github", "gitlab", "jira",
        "agile", "scrum", "kanban", "html", "css", "sass", "less", "bootstrap",
        "jquery", "typescript", "redux", "graphql", "rest", "soap", "json", "xml",
        "linux", "unix", "windows", "macos", "android", "ios", "flutter", "react native",
        "machine learning", "deep learning", "nlp", "computer vision", "data science",
        "data analysis", "data visualization", "big data", "hadoop", "spark", "kafka",
        "tableau", "power bi", "excel", "word", "powerpoint", "photoshop", "illustrator",
        "figma", "sketch", "adobe xd", "ui/ux", "responsive design", "seo", "sem",
        "digital marketing", "content marketing", "social media marketing", "email marketing",
        "project management", "product management", "leadership", "teamwork", "communication",
        "problem solving", "critical thinking", "creativity", "time management", "organization"
    ]
    
    doc = nlp(text)
    
    skills = []
    
    for skill in common_skills:
        if re.search(r'\b' + re.escape(skill) + r'\b', text.lower()):
            skills.append(skill)
    
    for ent in doc.ents:
        if ent.label_ in ["ORG", "PRODUCT"]:
            # Check if the entity is likely a skill (e.g., programming language, tool)
            if ent.text.lower() in common_skills:
                skills.append(ent.text.lower())
    
    skills = sorted(list(set(skills)))
    
    return skills

print(extract_skills(ntext))

['bootstrap', 'figma', 'firebase', 'git', 'github', 'javascript', 'leadership', 'photoshop', 'php', 'python', 'react', 'sql', 'teamwork', 'typescript']


that looks pretty accurate.

### Education

In [70]:
def extract_education(text):
    education_info = []

    education_section = ""
    blocks = re.split(r"\n{2,}|\r\n{2,}", text)
    for block in blocks:
        if "education" in block.lower():
            education_section = block
            break

    if not education_section:
        education_section = text

    lines = education_section.split("\n")
    degree_keywords = [
        "bachelor", "master", "phd", "mba", "b.tech", "m.tech", "b.sc", "m.sc", "cbse",
        "b tech", "m tech", "b.a", "m.a", "b.com", "m.com"
    ]

    year_pattern = r"(19|20)\d{2}"
    
    for line in lines:
        line_lower = line.lower()
        if any(deg in line_lower for deg in degree_keywords):
            degree_match = re.search(r"(b\.?\s?tech|m\.?\s?tech|bachelor(?:'s)?|master(?:'s)?|ph\.?d\.?|mba|b\.?sc\.?|m\.?sc\.?|cbse|b\.?a\.?|m\.?a\.?|b\.?com|m\.?com)", line_lower)
            year_match = re.findall(year_pattern, line)
            degree = degree_match.group(0).upper() if degree_match else "Unknown Degree"

            institution = "Unknown Institution"
            doc = nlp(line)
            for ent in doc.ents:
                if ent.label_ == "ORG" and any(k in ent.text.lower() for k in ["university", "college", "institute", "school", "academy", "cbse"]):
                    institution = ent.text
                    break
            else:
                institution_guess = " ".join(line.split()[:6])  # Take the first few words
                institution = institution_guess

            education_info.append({
                "degree": degree.title(),
                "institution": institution,
                "year": re.search(year_pattern, line).group(0) if re.search(year_pattern, line) else None
            })

    return education_info

print(extract_education(ntext))

[{'degree': 'Ma', 'institution': 'education srm institute of science and technology chennai b tech', 'year': '2024'}]


ok i tried but its really hard, at least i got the srm and btech

In [71]:
def extract_experience(text):
    doc = nlp(text)
    
    experience_keywords = [
        "experience", "work", "employment", "job", "position", "role", "career",
        "professional", "occupation", "worked", "employed", "intern", "internship"
    ]
    
    job_title_patterns = [
        r"(?i)(software engineer|software engineering intern|developer|programmer|analyst|designer|architect|manager|director|lead|consultant|specialist|administrator|technician|officer|coordinator|assistant|associate|senior|junior|chief|head|vp|vice president|ceo|cto|cio|cfo|president|founder|co-founder)"
    ]
    
    duration_pattern = r"(?i)(?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)\s+\d{4}\s+(?:to|–|-)\s+(?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)\s+\d{4}|(?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sep(?:tember)?|oct(?:ober)?|nov(?:ember)?|dec(?:ember)?)\s+\d{4}\s+(?:to|–|-)\s+present|(?:19|20)\d{2}\s+(?:to|–|-)\s+(?:19|20)\d{2}|(?:19|20)\d{2}\s+(?:to|–|-)\s+present|\d+\s+years?|\d+\s+months?"
    
    experience_sections = []
    
    paragraphs = text.split("\n\n")
    
    for paragraph in paragraphs:
        if any(keyword in paragraph.lower() for keyword in experience_keywords):
            experience_sections.append(paragraph)
    
    experience_info = []
    
    for section in experience_sections:
        title = "Job Title"
        for pattern in job_title_patterns:
            match = re.search(pattern, section)
            if match:
                title = match.group(0)
                break
        
        company = "Company"
        for ent in nlp(section).ents:
            if ent.label_ == "ORG":
                if not any(keyword in ent.text.lower() for keyword in ["university", "college", "institute", "school"]):
                    company = ent.text
                    break
        
        duration = None
        duration_match = re.search(duration_pattern, section)
        if duration_match:
            duration = duration_match.group(0)
        
        experience_info.append({
            "title": title,
            "company": company,
            "duration": duration
        })
    
    if not experience_info:
        # Extract job title
        title = "Job Title"
        for pattern in job_title_patterns:
            match = re.search(pattern, text)
            if match:
                title = match.group(0)
                break
        
        company = "Company"
        for ent in doc.ents:
            if ent.label_ == "ORG":
                if not any(keyword in ent.text.lower() for keyword in ["university", "college", "institute", "school"]):
                    company = ent.text
                    break
        
        duration = None
        duration_match = re.search(duration_pattern, text)
        if duration_match:
            duration = duration_match.group(0)
        
        experience_info.append({
            "title": title,
            "company": company,
            "duration": duration
        })
    
    return experience_info
print(extract_experience(yeshi))

[{'title': 'Developer', 'company': 'SRMIST', 'duration': None}, {'title': 'Head', 'company': 'HeadStarter AI', 'duration': 'Jul 2024 – Sep 2024'}, {'title': 'Software Engineer', 'company': 'Tiger Analytics', 'duration': 'Apr 2024 – Jul 2024'}, {'title': 'Software Engineer', 'company': 'Rotork', 'duration': 'Mar 2024 – Apr 2024'}, {'title': 'Lead', 'company': 'Company', 'duration': None}, {'title': 'Lead', 'company': 'Creative & Soft Skills', 'duration': None}]


OoO titles are a bit vague like head and lead but it got all the company names right like Headstarter, Tiger Analytics and Rotork, though it does have some false positives here and there.

### all of it together

In [74]:
def extract_resume_info(text):
    preprocessed_text = preprocess_text(text)
    
    skills = extract_skills(preprocessed_text)
    
    education = extract_education(text)
    
    experience = extract_experience(text)
    
    resume_info = {
        "skills": skills,
        "education": education,
        "experience": experience,
        "text": preprocessed_text
    }
    
    return resume_info

print(extract_resume_info(ntext))

{'skills': ['bootstrap', 'figma', 'firebase', 'git', 'github', 'javascript', 'leadership', 'photoshop', 'php', 'python', 'react', 'sql', 'teamwork', 'typescript'], 'education': [{'degree': 'Ma', 'institution': 'education srm institute of science and technology chennai b tech', 'year': '2024'}], 'experience': [{'title': 'developer', 'company': 'cse', 'duration': None}], 'text': 'yeshaswi prakash 91 90254 16053 yeshaswiprakash com linkedin com in yeshaswi github com yeshaswi profile summary software developer 1 yoe 4 hackathon winner 3rd year cse student at srmist experienced in building ai and ml powered applications leading teams and contributing to impactful projects actively involved in tech communities and campus leadership roles experience headstarter ai remote software engineering fellow jul 2024 sep 2024 built ai applications using next js openai pinecone and stripe api led a team of 3 engineering fellows mentored by engineers from amazon bloomberg and capital one tiger analytics